# Create a 20 Questions Answer with Rigging
- Claude 3 Version!

In [1]:
import jupyter_black

jupyter_black.load()

# Question Asker Agent

In [2]:
import rigging as rg

import ipywidgets as widgets
from IPython.display import display, clear_output


class AskerQuestion(rg.Model):
    question: str

In [3]:
BASE_MODEL = "claude-3-sonnet-20240229"

In [4]:
questions = []
answers = []


def ask_next_question(questions, answers, verbose=False):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to ask questions.
            - These questions should try to quickly find the keyword.
            - You will recieve back only yes or no responses.
            - Note previous questions and their answers when creating your next question
            - Do not assume anything specific too soon. Ask broad questions that could divide all possible remaining things into an even split.
            - The keyword will be a person, place or thing.
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}
        
        Ask your question within the tag {AskerQuestion.xml_tags()}
    """
    asker = (
        rg.get_generator(BASE_MODEL)
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )

    question = asker.last.parse(AskerQuestion).question
    if verbose:
        print(f"=== Question {len(questions) + 1} ====")
        print(question)

    return question

In [5]:
question = ask_next_question(questions, answers)

In [6]:
print(question)

Is the keyword a living thing?


# LLM Trying to Guess the Keyword "TOMATO"

In [7]:
answer = "no"
questions.append(question)
answers.append(answer)
question = ask_next_question(questions, answers)

In [8]:
answer = "no, its a red fruit used in sauses with spagetti"
questions.append(question)
answers.append(answer)
question = ask_next_question(questions, answers)

# Create our Answerer Agent

In [9]:
class YesNoAnswer(rg.Model):
    "Yes/No answer answer with coercion"
    answer: str


def get_agent_answer(keyword, question):

    system_prompt = f"""
        You are playing the game 20 questions. You will be asked a question an must answer yes or no.
        The keyword you are answering for is {keyword}
    """

    user_prompt = f"""
        The question is <question>{question}</question>
    
        Your response should be given between the tags {YesNoAnswer.xml_tags()}
    
        It should only be 'yes' or 'no'
        
        """

    answerer = (
        rg.get_generator(BASE_MODEL)
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )

    answer = answerer.last.parse(YesNoAnswer).answer
    return answer

In [10]:
keyword = "Tom Hanks"
question = "Is he a handsome lad?"
get_agent_answer(keyword, question)

'yes'

# Guesser Agent

In [11]:
class Guess(rg.Model):
    "20 Questions guess"
    guess: str


def make_guess(questions: list, answers: list, guesses: list, verbose=False):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to guess the keyword.
            - Note previous questions and their answers when creating your guess
            - The keyword will be a person, place or thing.
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}

        Previous Guesses are, do not repeat these:
            {','.join(guesses)}
        
        You will now guess for the 20 question game.
          - Be specific.
          - Guess only a single thing.
        
          
        Put the word for your guess in the following tag {Guess.xml_tags()}
    """
    asker = (
        rg.get_generator(BASE_MODEL)
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )

    guess = asker.last.parse(Guess).guess
    if verbose:
        print(f"=== Guess {len(questions) + 1} ====")
        print(guess)

    return guess

# Lets let the Agents fight it out!!!

In [12]:
import string


def keyword_guessed(guess: str, keyword: str) -> bool:
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)

    if normalize(guess) == normalize(keyword):
        return True
    # for s in alts:
    #   if normalize(s) == normalize(guess):
    #     return True

    return False

In [13]:
keyword = 'Tomato'
questions = []
answers = []
guesses = []
max_questions = 20
for i in range(max_questions):
    question = ask_next_question(questions, answers)
    answer = get_agent_answer(keyword, question)
    print('=' * 20)
    print(f'Round {i}')
    print(f'[Question]: {question}')
    print(f'[Answer]:   {answer}')
    questions.append(question)
    answers.append(answer)
    guess = make_guess(questions, answers, guesses)
    guesses.append(guess)
    print(f'[Guess]: {guess}')
    if keyword_guessed(guess, keyword):
        print('FOUND IT!!!! BIZNATCH!!!!!!')
        break

Round 0
[Question]: Is the keyword a living thing?
[Answer]:   yes
[Guess]: animal
Round 1
[Question]: Is the keyword an animal?
[Answer]:   no
[Guess]: human
Round 2
[Question]: Is the keyword a plant?
[Answer]:   yes
[Guess]: Tree
Round 3
[Question]: Is the keyword a tree?
[Answer]:   no
[Guess]: Flower
Round 4
[Question]: Is the keyword a flowering plant?
[Answer]:   yes
[Guess]: Rose
Round 5
[Question]: Is the keyword a vegetable?
[Answer]:   no
[Guess]: Tulip
Round 6
[Question]: Is the keyword a fruit?
[Answer]:   yes
[Guess]: Apple
Round 7
[Question]: Is the keyword a tropical fruit?
[Answer]:   no
[Guess]: Grape
Round 8
[Question]: Is the keyword a citrus fruit?
[Answer]:   no
[Guess]: Strawberry
Round 9
[Question]: Is the keyword a berry?
[Answer]:   yes
[Guess]: Blueberry
Round 10
[Question]: Is the keyword a strawberry?
[Answer]:   no
[Guess]: Raspberry
Round 11
[Question]: Is the keyword a blueberry?
[Answer]:   no
[Guess]: Blackberry
Round 12
[Question]: Is the keyword a ra

RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"Number of requests has exceeded your per-minute rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please try again later or contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}

# Can Claude Guess Pizza in 20 Questions?!

In [16]:
from time import sleep

In [17]:
keyword = "Pizza"
questions = []
answers = []
guesses = []
max_questions = 20

for i in range(max_questions):
    sleep(5)
    question = ask_next_question(questions, answers)
    answer = get_agent_answer(keyword, question)
    print("=" * 20)
    print(f"Round {i+1}")
    print(f"[Question]: {question}")
    print(f"[Answer]:   {answer}")
    questions.append(question)
    answers.append(answer)
    guess = make_guess(questions, answers, guesses)
    guesses.append(guess)
    print(f"[Guess]: {guess}")
    if keyword_guessed(guess, keyword):
        print("FOUND IT!!!! BIZNATCH!!!!!!")
        break

Round 1
[Question]: Is the keyword a living thing?
[Answer]:   no
[Guess]: Mountain
Round 2
[Question]: Is the keyword a physical object?
[Answer]:   yes
[Guess]: Smartphone
Round 3
[Question]: Is the keyword something man-made?
[Answer]:   yes
[Guess]: Computer
Round 4
[Question]: Is the keyword something portable or movable?
[Answer]:   yes
[Guess]: Laptop
Round 5
[Question]: Is the keyword some kind of tool or utensil?
[Answer]:   no
[Guess]: Car
Round 6
[Question]: Is the keyword some kind of electronic device?
[Answer]:   no
[Guess]: Suitcase
Round 7
[Question]: Is the keyword a vehicle?
[Answer]:   no
[Guess]: Bicycle
Round 8
[Question]: Is the keyword some kind of toy or game?
[Answer]:   no
[Guess]: Book
Round 9
[Question]: Is the keyword a piece of furniture?
[Answer]:   no
[Guess]: Guitar
Round 10
[Question]: Is the keyword some kind of clothing or accessory?
[Answer]:   no
[Guess]: Backpack
Round 11
[Question]: Is the keyword some kind of container or vessel?
[Answer]:   no


# Tom HANKS

In [18]:
keyword = "Tom Hanks"
questions = []
answers = []
guesses = []
max_questions = 20

for i in range(max_questions):
    sleep(5)
    question = ask_next_question(questions, answers)
    answer = get_agent_answer(keyword, question)
    print("=" * 20)
    print(f"Round {i+1}")
    print(f"[Question]: {question}")
    print(f"[Answer]:   {answer}")
    questions.append(question)
    answers.append(answer)
    guess = make_guess(questions, answers, guesses)
    guesses.append(guess)
    print(f"[Guess]: {guess}")
    if keyword_guessed(guess, keyword):
        print("FOUND IT!!!! BIZNATCH!!!!!!")
        break

Round 1
[Question]: Is the keyword a person?
[Answer]:   yes
[Guess]: Abraham Lincoln
Round 2
[Question]: Is this person a famous historical figure?
[Answer]:   no
[Guess]: Mom
Round 3
[Question]: Is this person alive today?
[Answer]:   yes
[Guess]: Barack Obama
Round 4
[Question]: Is this person a celebrity or public figure?
[Answer]:   yes
[Guess]: Elon Musk
Round 5
[Question]: Is this person involved in entertainment or media?
[Answer]:   yes
[Guess]: Taylor Swift
Round 6
[Question]: Is this person an actor or actress?
[Answer]:   yes
[Guess]: Jennifer Lawrence
Round 7
[Question]: Is this person primarily known for films or television?
[Answer]:   yes
[Guess]: Tom Hanks
FOUND IT!!!! BIZNATCH!!!!!!
